In [1]:
include("../src/geqrf.jl")
using Random
rng = MersenneTwister(2018)
import LinearAlgebra.dot
import LinearAlgebra.norm
import LinearAlgebra.cond
import LinearAlgebra.triu
import LinearAlgebra.qr
import LinearAlgebra.UniformScaling
;

In [2]:
n = 4
x = rand(rng, n)

4-element Array{Float64,1}:
 0.6545394330653942 
 0.10890604479083299
 0.7235855808911327 
 0.7203666853246062 

In [3]:
beta, v = house(x)

(0.4386315945756382, [1.87224, 0.108906, 0.723586, 0.720367])

In [4]:
n = length(x)
y = zeros(n); y[1] = norm(x)
Px = x - beta * dot(v,x) * v
if Px[1] < 0
    Px = -Px
end
@show Px
@show y
@show norm(Px - y)

Px = [1.2177, -0.0, -0.0, -0.0]
y = [1.2177, 0.0, 0.0, 0.0]
norm(Px - y) = 0.0


0.0

In [5]:
n = 4
e1 = zeros(n); e1[1] = 1.0
e1'

1×4 LinearAlgebra.Adjoint{Float64,Array{Float64,1}}:
 1.0  0.0  0.0  0.0

In [6]:
# Insert Julia code for exercise here

In [7]:
# Testing QR factorization using Householder transformations
n = 4

# Building an orthogonal matrix Q
Q = zeros(n,n)
for j=0:n-1, i=0:n-1
    Q[i+1,j+1] = cos(π*(2i+1)*j/2n)
end
for j=1:n
    Q[:,j] /= norm(Q[:,j])
end

# Initializing an upper triangular matrix R
R = triu(Float64[ i/j for i=1:n, j=1:n ])

# Matrix A
A = Q*R

4×4 Array{Float64,2}:
 0.5   0.903281    1.10219    1.09724  
 0.5   0.520598   -0.152935  -0.767982 
 0.5  -0.0205981  -0.513732   0.267982 
 0.5  -0.403281    0.231146  -0.0972388

In [8]:
# QR factorization
geqrf!(A)
# The factor R is stored in the upper triangular part of A

# These matrices should match up to the sign of each row
@show R,triu(A)

(R, triu(A)) = ([1.0 0.5 0.333333 0.25; 0.0 1.0 0.666667 0.5; 0.0 0.0 1.0 0.75; 0.0 0.0 0.0 1.0], [-1.0 -0.5 -0.333333 -0.25; 0.0 -1.0 -0.666667 -0.5; 0.0 0.0 1.0 0.75; 0.0 0.0 0.0 -1.0])


([1.0 0.5 0.333333 0.25; 0.0 1.0 0.666667 0.5; 0.0 0.0 1.0 0.75; 0.0 0.0 0.0 1.0], [-1.0 -0.5 -0.333333 -0.25; 0.0 -1.0 -0.666667 -0.5; 0.0 0.0 1.0 0.75; 0.0 0.0 0.0 -1.0])

In [9]:
# Illustrating the accuracy of QR factorizations using Householder transformations
n = 64

rng = MersenneTwister(2018)
Q, R = qr(rand(rng,n,n))
R = triu(rand(rng,n,n))

# Matrix A
A = Q*R

A0 = copy(A)
# Appending I as a way to get Q
A0 = hcat(A,UniformScaling(1.0))
# QR factorization
geqrf!(A0)
# The factor R is stored in the upper triangular part of A0
R0 = triu(A0[:,1:n])
# Q^T is stored in column n+1 to 2n
Q0 = A0[:,n+1:2*n]'

# Fixing the sign
for j=1:n
    if R0[j,j] * R[j,j] < 0
        R0[j,:] = -R0[j,:]
        Q0[:,j] = -Q0[:,j]
    end
end

condA = cond(A)
back_err = norm(Q0 * R0 - A)

# Accuracy of Q and R
@show norm(Q0' * Q0 - UniformScaling(1.0))   # Orthogonality
@show norm(R - R0)              # Forward error
@show back_err                  # Backward error
@show condA                     # Condition number of A
@show 1e-4 * condA * back_err   # Predicted forward error

norm(Q0' * Q0 - UniformScaling(1.0)) = 8.678550884996376e-15
norm(R - R0) = 8.912549009342026e-9
back_err = 2.3380444056710806e-14
condA = 5.453654884464741e9
0.0001 * condA * back_err = 1.275088729308355e-8


1.275088729308355e-8

In [10]:
# Systematic testing

n_test = 32:128
err = zeros(4,length(n_test))
k = 1

for n=n_test
    rng = MersenneTwister(2018 + n)
    Q, R = qr(rand(rng,n,n))
    R = triu(rand(rng,n,n))

    # Matrix A
    A = Q*R
    A0 = hcat(A,UniformScaling(1.0))
    
    # QR factorization
    geqrf!(A0)
    R0 = triu(A0[:,1:n])
    Q0 = A0[:,n+1:2*n]'

    # Fixing the sign
    for j=1:n
        if R0[j,j] * R[j,j] < 0
            R0[j,:] = -R0[j,:]
            Q0[:,j] = -Q0[:,j]
        end
    end

    condA = cond(A)
    back_err = norm(Q0 * R0 - A)

    # Saving values for plotting
    err[1,k] = norm(Q0' * Q0 - UniformScaling(1.0))   # Orthogonality
    err[2,k] = norm(R - R0)                           # Forward error
    err[3,k] = back_err                               # Backward error
    err[4,k] = 1e-4*condA*back_err                    # Predicted forward error
    k = k+1
end

In [11]:
using PlotlyJS

HTML{String}("    <script class='js-collapse-script'>\n        var curMatch =\n            window.location.href\n            .match(/(.*?)\\/notebooks\\/.*\\.ipynb/);\n\n        curMatch = curMatch ||\n            window.location.href\n            .match(/(.*?)\\/apps\\/.*\\.ipynb/);\n\n        if ( curMatch ) {\n            \$('head').append('<base href=\"' + curMatch[1] + '/\">');\n        }\n    </script>\n")

HTML{String}("<script class='js-collapse-script' src='/assetserver/f562d6507acc94a42cdf3b074249b311a179837e-assets/webio/dist/bundle.js'></script>")

HTML{String}("<script class='js-collapse-script' src='/assetserver/f562d6507acc94a42cdf3b074249b311a179837e-assets/providers/ijulia_setup.js'></script>")

HTML{String}("  <script class='js-collapse-script'>\n    \$('.js-collapse-script').parent('.output_subarea').css('padding', '0');\n  </script>\n")

In [12]:
t1 = scatter(x=1:n, y=err[1,:], name="Orthogonality")
t2 = scatter(x=1:n, y=err[2,:], name="Forward Error")
t3 = scatter(x=1:n, y=err[3,:], name="Backward Error")
t4 = scatter(x=1:n, y=err[4,:], name="Predicted Forward Error")
l = Layout(yaxis_type="log", xaxis_title="Matrix size", yaxis_title="Error")
# l[:height] = 300
# l[:width] = 450
plot([t1, t2, t3, t4], l)

data: [
  "scatter with fields name, type, x, and y",
  "scatter with fields name, type, x, and y",
  "scatter with fields name, type, x, and y",
  "scatter with fields name, type, x, and y"
]

layout: "layout with fields margin, xaxis, and yaxis"